In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
import os 
from dotenv import load_dotenv
from time import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
#loader = DirectoryLoader('PDF_Testing', glob="./*.pdf", loader_cls=PyPDFLoader)
loader = PyPDFLoader('How AI is expanding art history.pdf')
documents = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
texts = text_splitter.split_documents(documents)

In [4]:
len(texts)

8

In [5]:
texts[5]

Document(page_content='(quantification of differences in 3D shapes at \nvarious sizes and pose angles), optical-ray trac -\ning and other computational techniques have \nsystematically overturned Hockney’s theory \nmuch more conclusively than have arguments \nput forth by other scholars using conventional \nart-historical methods. \nRecovering lost cultural heritage\nComputer methods have also recovered \nmissing attributes or portions of incomplete \nartworks, such as the probable style and col -\nours of ghost paintings — works that have been \npainted over and are later revealed by imaging \nin X-rays or infrared radiation — such as Two \nWrestlers by Vincent van Gogh. This painting, \nfrom before 1886, was mentioned by the artist \nin a letter but considered lost until it was found \nbeneath another in 2012. Neural networks, trained on images and \ntext data, have also been used to recover the \nprobable colours of parts of Gustav Klimt’s \nlost ceiling painting, Medicine (see go.n

In [6]:
_ = load_dotenv()

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

llm=HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta", 
    model_kwargs={"temperature":0.2, "max_length":256},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
    )

In [7]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="BAAI/bge-base-en-v1.5",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [8]:
%%time
persist_directory = 'db_HuggingFace'

embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

CPU times: total: 406 ms
Wall time: 1.31 s


In [9]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
docs = retriever.get_relevant_documents("What is paranoia?")

In [10]:
len(docs)

2

In [11]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [12]:
def process_llm_response(qa_chain, query):
    print(f"Query: {query}\n")
    time_1 = time()
    llm_response = qa_chain(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print(f"\nResult:", llm_response['result'])
    print(f"\nmetadata:", llm_response['source_documents'][0].metadata)

In [13]:
query = "How does AI affect Art?"
process_llm_response(qa_chain, query)

Query: How does AI affect Art?

Inference time: 7.891 sec.

Result:  In order to fully harness the potential of AI in the study of art, we need to have a large collection of art images and accompanying information readily available online, and increased funding can help accelerate this process. AI can assist in analyzing and interpreting art, providing new insights and perspectives that were previously difficult to discern. It can also aid in the creation of new art, generating original works based on existing styles and techniques. However, there are also concerns about the potential for AI to replace human artists and the impact of AI-generated art on the traditional art market. As with any emerging technology, it is important to approach AI in art with a critical and thoughtful perspective, balancing the benefits and drawbacks.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [14]:
# break it down
query = "How does AI mimic human artwork?"
process_llm_response(qa_chain, query)

Query: How does AI mimic human artwork?

Inference time: 0.498 sec.

Result:  AI-driven tools are revealing how artists' understanding of the science of color and style contributes to their unique artistic style, allowing for the creation of AI-generated artwork that mimics the style of human artists. This is made possible by the same foundations required for the full power of AI in the study of art.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [15]:
query = "How do they measure Momentary social anxiety?"
process_llm_response(qa_chain, query)

Query: How do they measure Momentary social anxiety?

Inference time: 0.503 sec.

Result:  The study assessed momentary social anxiety using three specific items suggested in a previous study. Unfortunately, the specific items are not provided in this context, so the exact measurement method is not clear. However, it can be inferred that these items likely focus on measuring the intensity and duration of social anxiety in specific situations or interactions. Without further information, it is impossible to provide a more detailed answer.

metadata: {'page': 5, 'source': 'PDF_file\\s41598-023-47912-0.pdf'}


In [16]:
query = "Please give me an example of using AI to produce an artwork"
process_llm_response(qa_chain, query)

Query: Please give me an example of using AI to produce an artwork

Inference time: 55.912 sec.

Result:  While AI has the potential to revolutionize the way we create and appreciate art, it is still in its early stages of development. Currently, AI is primarily used to assist in the creation of art, rather than to produce fully autonomous works. For example, AI can be used to generate sketches or color palettes based on a specific style or theme, or to help artists identify patterns or anomalies in their work. However, fully autonomous AI-generated art is still a topic of ongoing research and debate in the art and technology communities. Some artists and critics argue that AI-generated art lacks the emotional and intellectual depth of human-made art, while others see it as a new and exciting frontier in the evolution of art. As AI technology continues to advance, it will be interesting to see how it is integrated into the art world and how it affects the way we create, appreciate, and

In [17]:
query = "How does machine learning and deep neural network help developing AI in art field?"
process_llm_response(qa_chain, query)

Query: How does machine learning and deep neural network help developing AI in art field?

Inference time: 0.535 sec.

Result:  Machine learning and deep neural networks are crucial foundations for realizing the full potential of AI in the study of art. By analyzing vast amounts of visual data, these technologies can help us understand the underlying principles of color, composition, and style in art. AI-driven tools are already revealing new insights into how artists' understanding of the science of color and style evolves over time, and how these principles are applied in different artistic traditions. As these technologies continue to advance, they will enable us to create more sophisticated and nuanced AI systems for generating, analyzing, and interpreting art.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [18]:
query = "What is By-eye art analysis?"
process_llm_response(qa_chain, query)

Query: What is By-eye art analysis?

Inference time: 3.558 sec.

Result:  By-eye art analysis refers to the traditional method of examining and evaluating works of art using visual observation and interpretation. It involves looking at the colors, composition, and other visual elements of a piece to determine its authenticity, quality, and value. This type of analysis is being challenged by new technologies that aim to automate and enhance this process through the use of artificial intelligence and other advanced techniques.

metadata: {'page': 0, 'source': 'How AI is expanding art history.pdf'}


In [19]:
query = "Other than produce Artwork, what else can AI use in Art study?"
process_llm_response(qa_chain, query)

Query: Other than produce Artwork, what else can AI use in Art study?

Inference time: 2.944 sec.

Result:  AI can be used to analyze and interpret art pieces, identify styles and techniques, and even assist in art conservation and restoration efforts by analyzing the chemical composition and condition of the artwork. It can also help in the creation of virtual exhibitions and tours, allowing people to experience art in new and immersive ways.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [20]:
query = "Give me an example of AI help recovering old art pieces"
process_llm_response(qa_chain, query)

Query: Give me an example of AI help recovering old art pieces

Inference time: 5.058 sec.

Result:  One example of AI being used to recover old art pieces is the project called "The Reconstruction of the Fresco Cycle in the Church of San Francesco in Borgo San Sepolcro" in Italy. This project used AI to analyze the remaining fragments of a fresco cycle painted by the Renaissance artist Benozzo Gozzoli in the 15th century. The AI was able to reconstruct the missing parts of the frescoes and create a digital restoration of the entire cycle, allowing scholars and the public to appreciate this lost cultural heritage in a new way.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [21]:
query = "What is the future of AI in art study?"
process_llm_response(qa_chain, query)

Query: What is the future of AI in art study?

Inference time: 5.087 sec.

Result:  The future of AI in art study is promising, as the availability of more art images and supporting information online and enlightened funding could accelerate the realization of the full power of AI in this field. However, it's important to note that while AI can assist in the analysis and interpretation of art, it cannot replace the critical thinking and subjective interpretation that is inherent in art appreciation and scholarship. Therefore, AI should be seen as a tool to enhance, rather than replace, human expertise in the study of art.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [22]:
query = "Is there any challenge that AI facing in art study?"
process_llm_response(qa_chain, query)

Query: Is there any challenge that AI facing in art study?

Inference time: 0.53 sec.

Result:  Yes, in order to fully harness the power of AI in the study of art, we need to establish the same foundations as other fields of perception and introduce new approaches and classes of questions to art scholarship. This presents a challenge as it requires a deep understanding of both the technical aspects of AI and the nuances of art history and criticism. Additionally, there is a need to address issues of data availability, privacy, and intellectual property in the context of AI and art.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [23]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x2a666dbb1d0>)

In [24]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:
